In [ ]:
#Rafael Vilela Santa Rosa
!pip install -q pvlib
import time
import pvlib
from pvlib import location, irradiance, tools
from pvlib.iotools import read_tmy3
import pandas as pd
from matplotlib import pyplot as plt
import pathlib
import numpy as np
from scipy.optimize import curve_fit
from scipy.signal import hilbert
from scipy.stats import skew, kurtosis, variation

tempo_inicial=(time.time())

#obs: arquivo csv escolhido não considera um ano bissexto

energia_hs = [70, 82.4, 122.9, 150.4, 166, 176.9, 174.9, 162.4, 123.8, 102.4, 68.2, 67.5]

df1 = pd.DataFrame({'energia_hs': energia_hs})

energia_pvsyst = [77.9, 88.8, 128.8, 159, 175, 189.1, 185.2, 171.9, 142, 109.1, 85.8, 66]

df2 = pd.DataFrame({'energia_pvsyst': energia_pvsyst})

# get full path to the data directory
DATA_DIR = pathlib.Path(pvlib.__file__).parent / 'data'

# get TMY3 dataset
tmy, metadata = read_tmy3(DATA_DIR / '723170TYA.CSV', coerce_year=2023)

# create location object to store lat, lon, timezone
location = location.Location.from_tmy(metadata)

# print metadata
for key, value in metadata.items():
    print(f'{key}: {value}')
latitude = metadata['latitude']
Φ = latitude
    
    
# calculate the necessary variables to do transposition.
times = pd.date_range(start='2023-01-01', end='2023-12-31 23:59:59', freq='1H')

# create a new DataFrame that only contains the necessary columns and that has a consistent time index
df = pd.DataFrame(index=times)
df['DHI'] = tmy['DHI'].values
df['DNI'] = tmy['DNI'].values
df['GHI'] = tmy['GHI'].values

max_value1 = df['DHI'].max()
print(f"Valor máximo DHI: {max_value1}")
max_value2 = df['DNI'].max()
print(f"Valor máximo DNI: {max_value2}")
max_value3 = df['GHI'].max()
print(f"Valor máximo GHI: {max_value3}")

solar_position = location.get_solarposition(times)

# calculate extra terrestrial radiation
dni_extra = pvlib.irradiance.get_extra_radiation(df.index)

print('/n')
Optimal_tilt_angle_NH = 1.3793 + Φ*(1.2011 + Φ*(-0.014404 + Φ*0.000080509))
print(f'Inclinação ótima:{Optimal_tilt_angle_NH}')


############################################### inicio

df_poa = pvlib.irradiance.get_total_irradiance(
                                surface_tilt=30, surface_azimuth=90,
                                dhi=df['DHI'], dni=df['DNI'],ghi=df['GHI'], dni_extra=dni_extra,
                                solar_zenith=solar_position['apparent_zenith'],
                                solar_azimuth=solar_position['azimuth'],
            airmass=pvlib.atmosphere.get_relative_airmass(solar_position['apparent_zenith']),
    model='Perez')

print("\n-----------Dados-----------")

df_poa['poa_global'].fillna(0, inplace=True)

# Calculate quartiles
Q1 = df_poa['poa_global'].quantile(0.25)
Q2 = df_poa['poa_global'].quantile(0.5)
Q3 = df_poa['poa_global'].quantile(0.75)
Q4 = df_poa['poa_global'].quantile(1)
P95 = df_poa['poa_global'].quantile(0.95)
P98 = df_poa['poa_global'].quantile(0.98)
P99 = df_poa['poa_global'].quantile(0.99)

print("First Quartile:", Q1)
print("Second Quartile (Median):", Q2)
print("Third Quartile:", Q3)
print("Fourth Quartile (Max):", Q4)
print("95th Percentile:", P95)
print("98th Percentile:", P98)
print("99th Percentile:", P99)
print("\n")


perez_zeros = df_poa['poa_global'][df_poa['poa_global'] == 0].count()
print(f"Number of zeros in Perez model: {perez_zeros}")

perez_high = df_poa['poa_global'][df_poa['poa_global'] > 1367].count()
print(f"Number of high numbers in Perez model: {perez_high}")

perez_mid = df_poa['poa_global'][df_poa['poa_global'] > P98].count()
print(f"Number of mid numbers (>P98) in Perez model: {perez_mid}")

skewness_list = []
envelope_list = []
kurtosis_list = []
cv_list = []
fs=1

# Loop por cada dia do ano
for day in pd.date_range('2023-01-01', '2023-12-31'):
    # Get the data for the current day
    day_data = df_poa.loc[day.strftime('%Y-%m-%d'), 'poa_global']
    
    # Calculate Hilbert transform of the data
    analytic_signal = hilbert(day_data)
    amplitude_envelope = np.abs(analytic_signal)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    instantaneous_frequency = (np.diff(instantaneous_phase) / (2.0*np.pi) * fs)
    
    # Calculate skewness and append to list
    skewness_list.append(skew(day_data))
    
    # Calculate mean envelope and append to list
    envelope_list.append(np.mean(amplitude_envelope))
    
    # Calculate kurtosis and append to list
    kurtosis_list.append(kurtosis(day_data))
    
    # Calculate coefficient of variation and append to list
    cv_list.append(variation(day_data))
    
mean_skewness = np.mean(skewness_list)
mean_envelope = np.mean(envelope_list)
mean_kurtosis = np.mean(kurtosis_list)
mean_cv = np.mean(cv_list)

print(f"Mean skewness: {mean_skewness}")
print(f"Mean envelope: {mean_envelope}")
print(f"Mean kurtosis: {mean_kurtosis}")
print(f"Mean Coefficient of Variation: {mean_cv}")


#print("\nFirst values of Perez model:")
#print(df_poa['poa_global'].head(4000))

# plot the results
df_poa['poa_global'].plot()
plt.ylabel('POA [W/m^2]')
plt.title('POA Irradiance using Perez Model')
plt.show()


# Para um mês específico:
specific_month = df_poa.loc['2023-06']

specific_month['poa_global'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance in 06/2023')
plt.show()


# Para um dia específico:
specific_day = df_poa.loc['2023-06-01']

specific_day['poa_global'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance on 2023-06-01')
plt.show()


############################################### calculo de energia


# Create a new column for the month
df_poa['month'] = df_poa.index.month

# Group by month and sum the irradiance to get the total irradiance for each month
monthly_irradiance = df_poa.groupby('month')['poa_global'].sum()

# The total irradiance is in units of W/m^2. 
monthly_energy = monthly_irradiance * 3600

# Convert the energy from Joules (J) to kilowatt-hours (kWh)
monthly_energy_kwh = monthly_energy / (1000 * 3600)


percentual1=((monthly_energy_kwh.values/ df1['energia_hs'].values) * 100)
percentual2=((monthly_energy_kwh.values/ df2['energia_pvsyst'].values) * 100)

df3 = pd.DataFrame({'percentual': percentual1})
df4 = pd.DataFrame({'percentual': percentual2})


print('valor médio HS:')
print(df3['percentual'].mean())
print('\nPercentual HelioScope:')
print(df3['percentual'])

print('valor médio PVsyst:')
print(df4['percentual'].mean())
print('\nPercentual PVsyst:')
print(df4['percentual'])


############################################### identificação de outlier

"""

# Create a new column for the day
df_poa['day'] = df_poa.index.dayofyear

# Group by day and sum the irradiance to get the total irradiance for each day
daily_irradiance = df_poa.groupby('day')['poa_global'].sum()

# The total irradiance is in units of W/m^2.
daily_energy = daily_irradiance * 3600

# Convert the energy from Joules (J) to kilowatt-hours (kWh)
daily_energy_kwh = daily_energy / (1000 * 3600)

"""

# Função para representar a forma senoidal
def sinusoidal(x, a, b, c, d):
    return a * np.sin(b * (x - np.radians(c))) + d

# Agregar os dados por dia
daily_irradiance = df_poa['poa_global'].resample('D').sum()

# Obter o dia do ano para usar como variável independente no ajuste
day_of_year = daily_irradiance.index.dayofyear

# Parâmetros iniciais para o ajuste: amplitude, período, fase horizontal, fase vertical
# O período é ajustado para 2*pi/365 para refletir o ciclo anual
# A fase horizontal é ajustada para colocar o pico no meio do ano (dia 182)
initial_parameters = [daily_irradiance.max(), 2 * np.pi / 365, 182, daily_irradiance.min()]

# Ajustar a função aos dados
parameters, _ = curve_fit(sinusoidal, day_of_year, daily_irradiance, p0=initial_parameters)

# Gerar um array de x para a função ajustada
x_fit = np.linspace(1, 365, 1000)

# Gerar os valores y da função ajustada
y_fit = sinusoidal(x_fit, *parameters)

print(f"A função senoidal de tendência é: {parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}")

# Plotar os dados originais e a função ajustada
plt.scatter(day_of_year, daily_irradiance, label='data')
plt.plot(x_fit, y_fit, color='red', label='fit')
plt.xlabel('Day of year')
plt.ylabel('Irradiação diária [Wh]')
plt.title('Sinusoidal fit to daily POA irradiance data')
plt.legend()
plt.text(170,0,f"{parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}",ha='center', va='center')
plt.show()

############################################### correção inicial de outlier

print("\n-----------Correção inicial outlier-----------")

max_value = df_poa['poa_global'].max()
print(f"Valor máximo antes: {max_value}")

df_poa['poa_global'] = df_poa['poa_global'].clip(upper=P98)

max_value = df_poa['poa_global'].max()
print(f"Valor máximo depois: {max_value}")


############################################### calculo de energia


# Create a new column for the month
df_poa['month'] = df_poa.index.month

# Group by month and sum the irradiance to get the total irradiance for each month
monthly_irradiance = df_poa.groupby('month')['poa_global'].sum()

# The total irradiance is in units of W/m^2. 
monthly_energy = monthly_irradiance * 3600

# Convert the energy from Joules (J) to kilowatt-hours (kWh)
monthly_energy_kwh = monthly_energy / (1000 * 3600)


percentual1=((monthly_energy_kwh.values/ df1['energia_hs'].values) * 100)
percentual2=((monthly_energy_kwh.values/ df2['energia_pvsyst'].values) * 100)


df3 = pd.DataFrame({'percentual': percentual1})
df4 = pd.DataFrame({'percentual': percentual2})


print('valor médio HS:')
print(df3['percentual'].mean())
print('\nPercentual HelioScope:')
print(df3['percentual'])

print('valor médio PVsyst:')
print(df4['percentual'].mean())
print('\nPercentual PVsyst:')
print(df4['percentual'])

############################################### identificação de outlier


# Função para representar a forma senoidal
def sinusoidal(x, a, b, c, d):
    return a * np.sin(b * (x - np.radians(c))) + d

# Agregar os dados por dia
daily_irradiance = df_poa['poa_global'].resample('D').sum()

# Obter o dia do ano para usar como variável independente no ajuste
day_of_year = daily_irradiance.index.dayofyear

# Parâmetros iniciais para o ajuste: amplitude, período, fase horizontal, fase vertical
# O período é ajustado para 2*pi/365 para refletir o ciclo anual
# A fase horizontal é ajustada para colocar o pico no meio do ano (dia 182)
initial_parameters = [daily_irradiance.max(), 2 * np.pi / 365, 182, daily_irradiance.min()]

# Ajustar a função aos dados
parameters, _ = curve_fit(sinusoidal, day_of_year, daily_irradiance, p0=initial_parameters)

# Gerar um array de x para a função ajustada
x_fit = np.linspace(1, 365, 1000)

# Gerar os valores y da função ajustada
y_fit = sinusoidal(x_fit, *parameters)

print(f"A função senoidal de tendência é: {parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}")

# Plotar os dados originais e a função ajustada
plt.scatter(day_of_year, daily_irradiance, label='data')
plt.plot(x_fit, y_fit, color='red', label='fit')
plt.xlabel('Day of year')
plt.ylabel('Irradiação diária [Wh]')
plt.title('Sinusoidal fit to daily POA irradiance data - limitação de potência pelo percentil 98')
plt.legend()
plt.text(170,0,f"{parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}",ha='center', va='center')
plt.show()


############################################### tratamento de outlier 

print("\n-----------Tratamento de outlier-----------")

'''
# Calcule os valores da função de tendência para cada dia do ano
trend_values = sinusoidal(day_of_year, *parameters)

# limite superior e inferior
upper_limit = trend_values * 1.15
lower_limit = trend_values * 1.05

#= série pandas para os valores de tendência e limite superior/inferior para facilitar a manipulação
trend_series = pd.Series(trend_values, index=daily_irradiance.index)
upper_limit_series = pd.Series(upper_limit, index=daily_irradiance.index)
lower_limit_series = pd.Series(lower_limit, index=daily_irradiance.index)

##

# Resample the hourly data to daily data
daily_poa_global = df_poa['poa_global'].resample('D').sum()

# Replace values in the daily data that are above the upper limit
daily_poa_global.where(daily_poa_global <= upper_limit_series, upper_limit_series, inplace=True)
daily_poa_global.where(daily_poa_global >= lower_limit_series, lower_limit_series, inplace=True)

# Resample the daily data back to hourly data
df_poa['poa_global'] = daily_poa_global.resample('H').ffill() / 24


# plot the results
df_poa['poa_global'].plot()
plt.ylabel('POA [W/m^2]')
plt.title('POA Irradiance using Perez Model')
plt.show()

# Para um mês específico:
specific_month = df_poa.loc['2023-06']

specific_month['poa_global'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance in 06/2023')
plt.show()


# Para um dia específico:
specific_day = df_poa.loc['2023-06-01']

specific_day['poa_global'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance on 2023-06-01')
plt.show()


'''

############################################### tratamento de outlier 2
# Create a new DataFrame to store the adjusted irradiance values
df_poa_adjusted = df_poa.copy()

daily_parameters = pd.DataFrame(columns=['a', 'b', 'c', 'd'], index=df_poa.index.normalize().unique())

coef_upper = 1.15
coef_lower = 1.05

# Criação de um DataFrame para armazenar os parâmetros diários
daily_parameters = pd.DataFrame(index=pd.date_range('2023-01-01', '2023-12-31'), 
                                columns=['amplitude', 'period', 'phase_shift', 'vertical_shift'])


# Loop por cada dia do ano
for day in pd.date_range('2023-01-01', '2023-12-31'):
    # Obter os dados para o dia atual
    day_data = df_poa.loc[day.strftime('%Y-%m-%d')]
    
    # Obter a hora do dia para usar como variável independente no ajuste
    hour_of_day = day_data.index.hour
    
    # Parâmetros iniciais para o ajuste: amplitude, período, fase horizontal, fase vertical
    # O período é ajustado para 2*pi para refletir o ciclo diário
    initial_parameters = [day_data['poa_global'].max(), 2 * np.pi, 12, day_data['poa_global'].min()]
    
    # Ajustar a função aos dados
    try:
        parameters, _ = curve_fit(sinusoidal, hour_of_day, day_data['poa_global'], p0=initial_parameters, maxfev=5000)
    except RuntimeError:
        print(f"Failed to fit curve for day {day}")
        continue
    # Armazenar os parâmetros no DataFrame
    daily_parameters.loc[day, ['amplitude', 'period', 'phase_shift', 'vertical_shift']] = parameters
    
# Loop por cada dia do ano novamente para ajustar os outliers
for day in pd.date_range('2023-01-01', '2023-12-31'):
    # Obter os dados para o dia atual
    day_data = df_poa.loc[day.strftime('%Y-%m-%d')]
    
    # Obter a hora do dia para usar como variável independente no ajuste
    hour_of_day = day_data.index.hour
    
    # Obter os parâmetros para o dia atual
    parameters = daily_parameters.loc[day]
    
    # Calcular os valores da função de tendência para cada hora do dia
    trend_values = sinusoidal(hour_of_day, *parameters)
    
    # Definir um limite superior e inferior
    upper_limit = trend_values * coef_upper
    lower_limit = trend_values * coef_lower
    
    # série pandas para os valores de tendência e limite superior/inferior para facilitar a manipulação
    trend_series = pd.Series(trend_values, index=day_data.index)
    upper_limit_series = pd.Series(upper_limit, index=day_data.index)
    lower_limit_series = pd.Series(lower_limit, index=day_data.index)
    
    for i in range(len(day_data)):
        if day_data['poa_global'].iloc[i] > 0:
        # Substituir valores nos dados diários      
            if day_data['poa_global'].iloc[i] > upper_limit_series.iloc[i]:
                day_data['poa_global'].iloc[i] = upper_limit_series.iloc[i]
            elif day_data['poa_global'].iloc[i] < lower_limit_series.iloc[i]:
                day_data['poa_global'].iloc[i] = lower_limit_series.iloc[i]
    
    # Armazenar os dados ajustados de volta no DataFrame original
    df_poa.loc[day.strftime('%Y-%m-%d'), 'poa_global'] = day_data['poa_global']

#  df['total_irradiance'] ajustado para outliers com base na tendência senoidal diária
#para valores negativos por conta do ajuste da senoide
df_poa['poa_global'] = df_poa['poa_global'].clip(lower=0)

print("\n-----------Termino do tratamento de outlier-----------")

# plot the results
df_poa['poa_global'].plot()
plt.ylabel('POA Irradiance [W/m^2]')
plt.title('POA Irradiance using Perez Model')
plt.show()

# Para um mês específico:
specific_month = df_poa.loc['2023-06']

specific_month['poa_global'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance in 06/2023')
plt.show()

# Para um dia específico:
specific_day = df_poa.loc['2023-06-01']

specific_day['poa_global'].plot()
plt.ylabel('Irradiance [W/m^2]')
plt.title('POA Irradiance on 2023-06-01')
plt.show()


# um dia específico
specific_day = "2023-06-01"

# irradiação para esse dia
day_data = df_poa.loc[specific_day]

# parâmetros para esse dia
parameters = daily_parameters.loc[specific_day]

# curva senoidal para esse dia
hour_of_day = np.arange(24)
sinusoidal_data = sinusoidal(hour_of_day, *parameters)

# irradiação e a curva senoidal
plt.figure(figsize=(10, 6))
plt.plot(day_data.index.hour, day_data['poa_global'], label='Irradiância')
plt.plot(hour_of_day, sinusoidal_data, label='Curva Senoidal', linestyle='--')
plt.xlabel('Hora do dia')
plt.ylabel('Irradiância [W/m²]')
plt.title(f'Irradiância e Curva Senoidal para o dia {specific_day}')
plt.legend()
plt.text(12,0,f"{parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}",ha='center', va='center')
plt.grid(True)
plt.show()


############################################### identificação de outlier

# Função para representar a forma senoidal
def sinusoidal(x, a, b, c, d):
    return a * np.sin(b * (x - np.radians(c))) + d

# Agregar os dados por dia
daily_irradiance = df_poa['poa_global'].resample('D').sum()

# Obter o dia do ano para usar como variável independente no ajuste
day_of_year = daily_irradiance.index.dayofyear

# Parâmetros iniciais para o ajuste: amplitude, período, fase horizontal, fase vertical
# O período é ajustado para 2*pi/365 para refletir o ciclo anual
# A fase horizontal é ajustada para colocar o pico no meio do ano (dia 182)
initial_parameters = [daily_irradiance.max(), 2 * np.pi / 365, 182, daily_irradiance.min()]

# Ajustar a função aos dados
parameters, _ = curve_fit(sinusoidal, day_of_year, daily_irradiance, p0=initial_parameters)

# Gerar um array de x para a função ajustada
x_fit = np.linspace(1, 365, 1000)

# Gerar os valores y da função ajustada
y_fit = sinusoidal(x_fit, *parameters)

print(f"A função senoidal de tendência é: {parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}")

# Plotar os dados originais e a função ajustada
plt.scatter(day_of_year, daily_irradiance, label='data')
plt.plot(x_fit, y_fit, color='red', label='fit')
plt.xlabel('Day of year')
plt.ylabel('Irradiância diária [Wh]')
plt.title('Sinusoidal fit to daily POA irradiance data - com tratamento de outliers')
plt.legend()
plt.text(170,0,f"{parameters[0]:.2f} * sin({parameters[1]:.2f} * (x - {np.degrees(parameters[2]):.2f})) + {parameters[3]:.2f}",ha='center', va='center')
plt.show()


###############################################calculo de energia


# Create a new column for the month
df_poa['month'] = df_poa.index.month

# Group by month and sum the irradiance to get the total irradiance for each month
monthly_irradiance = df_poa.groupby('month')['poa_global'].sum()

# The total irradiance is in units of W/m^2. 
monthly_energy = monthly_irradiance * 3600

# Convert the energy from Joules (J) to kilowatt-hours (kWh)
monthly_energy_kwh = monthly_energy / (1000 * 3600)

#monthly_energy_kwh

percentual3=((monthly_energy_kwh.values/ df1['energia_hs'].values) * 100)
percentual4=((monthly_energy_kwh.values/ df2['energia_pvsyst'].values) * 100)

df5 = pd.DataFrame({'percentual': percentual3})
df6 = pd.DataFrame({'percentual': percentual4})

print('valor médio HS:')
print(df5['percentual'].mean())
print('\nPercentual HelioScope:')
print(df5['percentual'])
print('valor médio PVsyst:')
print(df6['percentual'].mean())
print('\nPercentual PVsyst:')
print(df6['percentual'])

months = list(range(1, 13))
errors1 = np.abs(df3['percentual'] - 100)
errors2 = np.abs(df4['percentual'] - 100)
errors3 = np.abs(df5['percentual'] - 100)
errors4 = np.abs(df6['percentual'] - 100)


# Plot HelioScope data
plt.errorbar(months, df3['percentual'], yerr=errors1, fmt='o', color='blue', label='Original (em relação ao HelioScope)')

plt.errorbar(months, df5['percentual'], yerr=errors3, fmt='o', color='red', label='Com correção de outlier (em relação ao HelioScope)')

# linha em y=100
plt.axhline(100, color='black', linestyle='--')

plt.title('Comparação de percentuais mensais (Perez/HelioScope)')
plt.xlabel('Mês')
plt.ylabel('Percentual (%)')

plt.legend()

for month, error1, error3 in zip(months, errors1, errors3):
    plt.text(month, df3['percentual'][month-1], f"{error1:.1f}%", ha='center', va='bottom')
    plt.text(month, df5['percentual'][month-1], f"{error3:.1f}%", ha='center', va='top')

plt.legend()

# Exibir o gráfico
plt.show()


##############################

# Plot PVSyst data
plt.errorbar(months, df4['percentual'], yerr=errors2, fmt='o', color='yellow', label='Original (em relação ao PVsyst)')
7
plt.errorbar(months, df6['percentual'], yerr=errors4, fmt='o', color='green', label='Com correção de outlier (em relação ao PVsyst)')

# linha em y=100
plt.axhline(100, color='black', linestyle='--')

plt.title('Comparação de percentuais mensais (Perez/PVsyst)')
plt.xlabel('Mês')
plt.ylabel('Percentual (%)')

plt.legend()

for month, error2, error4 in zip(months, errors2, errors4):
    plt.text(month, df4['percentual'][month-1], f"{error2:.1f}%", ha='center', va='bottom')
    plt.text(month, df6['percentual'][month-1], f"{error4:.1f}%", ha='center', va='top')

plt.legend()

# Exibir o gráfico
plt.show()


skewness_list2 = []
envelope_list2 = []
kurtosis_list2 = []
cv_list2 = []

# Loop por cada dia do ano
for day in pd.date_range('2023-01-01', '2023-12-31'):
    # Get the data for the current day
    day_data = df_poa.loc[day.strftime('%Y-%m-%d'), 'poa_global']
    
    # Calculate Hilbert transform of the data
    analytic_signal = hilbert(day_data)
    amplitude_envelope = np.abs(analytic_signal)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    instantaneous_frequency = (np.diff(instantaneous_phase) / (2.0*np.pi) * fs)
    
    # Calculate skewness and append to list
    skewness_list2.append(skew(day_data))
    
    # Calculate mean envelope and append to list
    envelope_list2.append(np.mean(amplitude_envelope))
    
    # Calculate kurtosis and append to list
    kurtosis_list2.append(kurtosis(day_data))
    
    # Calculate coefficient of variation and append to list
    cv_list2.append(variation(day_data))
    
mean_skewness = np.mean(skewness_list2)
mean_envelope = np.mean(envelope_list2)
mean_kurtosis = np.mean(kurtosis_list2)
mean_cv = np.mean(cv_list2)

print(f"Mean skewness: {mean_skewness}")
print(f"Mean envelope: {mean_envelope}")
print(f"Mean kurtosis: {mean_kurtosis}")
print(f"Mean Coefficient of Variation: {mean_cv}")

tempo_final=(time.time())
tempo=tempo_final - tempo_inicial

print(f"{tempo} segundos")
print("Fim!")